In [2]:
# !pip install transformers
# !pip install git+https://github.com/yuchenlin/LLM-Blender.git
# !pip install "pydantic<2"
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import pandas as pd
import numpy as np
import torch
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast, AdamW, AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from torch import nn
from transformers import Trainer
import torch.nn.functional as F
import torchvision.transforms as tr # 이미지 전처리 기능들을 제공하는 라이브러리
from torch.utils.data import DataLoader, Dataset # 데이터를 모델에 사용할 수 있도록 정리해 주는 라이브러리
import csv

from llm_blender.pair_ranker.pairrm import DebertaV2PairRM # or copy the DebertaV2PairRM definition here, https://github.com/yuchenlin/LLM-Blender/blob/main/llm_blender/pair_ranker/pairrm.py
from transformers import AutoTokenizer
from typing import List

# CUDA 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.8/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain libcudart.so as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.8/dist-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//matplotlib_inline.backend_inline'), PosixPath('module')}
  warn(msg)


Using device: cuda


In [4]:
test_df = pd.read_csv("./data/test.csv")
test_df

,id,질문
0,TEST_000,"방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용..."
1,TEST_001,도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?
2,TEST_002,"큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?"
3,TEST_003,"철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있..."
4,TEST_004,도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?
...,...,...
125,TEST_125,분말 소화기를 사용할 때 주의해야 할 사항은 무엇인가요? 그리고 아파트 도배 평수를...
126,TEST_126,"압출법 보온판의 가장 큰 장점은 무엇인가요?"""
127,TEST_127,평지붕의 누수 문제를 방지하기 위해 수성 벽체용 탄성 방수 도료를 사용하는 것이 어...
128,TEST_128,석고수정이 발생하는 가장 큰 원인은 무엇인가요? 그리고 이를 해결하는 방법에 대해 ...


In [5]:
inputs = test_df['질문'].tolist()
inputs

['방청 페인트의 종류에는 어떤 것들이 있는지 알고 계신가요? 또한, 원목사이딩을 사용하는 것에 어떤 단점이 있을까요?',
 '도배지에 녹은 자국이 발생하는 주된 원인과 그 해결 방법은 무엇인가요?',
 '큐블럭의 단점을 알려주세요. 또한, 압출법 단열판을 사용하는 것의 장점은 무엇인가요?',
 '철골구조를 사용하는 고층 건물에서, 단열 효과를 높이기 위한 시공 방법은 무엇이 있을까요?',
 '도배지의 완전한 건조를 위해 몇 주 동안 기다려야 하나요?',
 '철근철골콘크리트 구조가 적용된 건물의 장단점은 무엇인가요?',
 '어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?',
 '도배지 들뜸 현상이 발생하는 가장 일반적인 원인은 무엇인가요?',
 '도배지에 얼룩이 생기는 다양한 원인들에 대해서 자세히 알려주세요.',
 '고층 건물을 건축할 때 철골구조가 주로 선택되는 이유는 무엇인가요?',
 '유성페인트의 환경 오염에 대한 예시를 알려주세요.',
 '훼손과 오염은 어떻게 다른가요? 또한, 부실 시공으로 인해 타공하자가 발생할 가능성이 있나요?',
 '철골콘크리트 구조에서 사용되는 철근의 재료적 특징은 무엇인가요?"',
 '방염벽지가 실크벽지와 합지벽지와 다른 점은 무엇인가요?',
 '도배지에 곰팡이가 생겼을 때 높은 습도가 원인인 경우, 습기 관리는 어떻게 해야 할까요?"',
 '롱브릭타일은 어떤 종류의 타일인가요? 그리고 페인트 상도재의 역할과 사용 방법에 대해 알려주세요.',
 '준불연재료는 무엇인가요? 그리고 유성페인트를 사용하는 것에 대한 부작용이 있을까요?',
 '어떤 환경 요인이 몰딩 수정을 유발할 수 있는가요? 그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요?',
 '벽에 뚫린 구멍이 이상하다고 하셨는데, 타공 불량이 발생하는 원인과 그 해결 방법에 대해 자세히 설명해주실 수 있을까요?"',
 '다이닝 룸을 더 고급스럽게 꾸밀려면 어떻게 

In [6]:
import re



def find_number_and_remainder(s):
    # Regular expression to match a number at the beginning of the string
    pattern = r'^(\d+)(.*)'
    
    # Find the match
    match = re.match(pattern, s)
    
    if match:
        # Extract the matched number and the rest of the string
        number_str, remainder = match.groups()
        
        # Convert the extracted number to an integer
        number = int(number_str)
        
        return number, remainder
    else:
        # Return None if no match is found
        return None, None



In [7]:
# Open the text file
with open('./result_skt_r.txt', 'r') as file:
    # Read all lines from the file and store them in a list
    lines = file.readlines()
# lines
skt_result = []
for line in lines:
    skt_result.append(line.strip())

with open('./result_skt_appendix_1.txt', 'r') as file:
    # Read all lines from the file and store them in a list
    lines = file.readlines()
# lines
result_1 = []
for line in lines:
    result_1.append(line.strip())
len(result_1)

with open('./result_skt_appendix_2.txt', 'r') as file:
    # Read all lines from the file and store them in a list
    lines = file.readlines()
# lines
result_2 = []
for line in lines:
    result_2.append(line.strip())

with open('./result_skt_appendix_3.txt', 'r') as file:
    # Read all lines from the file and store them in a list
    lines = file.readlines()
# lines
result_3 = []
for line in lines:
    result_3.append(line.strip())

for app in result_1:
    index, remain = find_number_and_remainder(app)
    skt_result[index] = skt_result[index] + ' ' + remain
for app in result_2:
    index, remain = find_number_and_remainder(app)
    skt_result[index] = skt_result[index] + ' ' + remain
for app in result_3:
    index, remain = find_number_and_remainder(app)
    skt_result[index] = skt_result[index] + ' ' + remain

In [8]:
# Open the text file
with open('./result_alpha.txt', 'r') as file:
    # Read all lines from the file and store them in a list
    lines = file.readlines()
# lines
alpha_result = []
for line in lines:
    alpha_result.append(line.strip())

with open('./result_alpha_appendix_1_r.txt', 'r') as file:
    # Read all lines from the file and store them in a list
    lines = file.readlines()
# lines
result_1 = []
for line in lines:
    result_1.append(line.strip())
len(result_1)

with open('./result_alpha_appendix_2_r.txt', 'r') as file:
    # Read all lines from the file and store them in a list
    lines = file.readlines()
# lines
result_2 = []
for line in lines:
    result_2.append(line.strip())

with open('./result_alpha_appendix_3_r.txt', 'r') as file:
    # Read all lines from the file and store them in a list
    lines = file.readlines()
# lines
result_3 = []
for line in lines:
    result_3.append(line.strip())

for app in result_1:
    index, remain = find_number_and_remainder(app)
    alpha_result[index] = alpha_result[index] + ' ' + remain
for app in result_2:
    index, remain = find_number_and_remainder(app)
    alpha_result[index] = alpha_result[index] + ' ' + remain
for app in result_3:
    index, remain = find_number_and_remainder(app)
    alpha_result[index] = alpha_result[index] + ' ' + remain

In [8]:
# # Open the text file
# #kko
# with open('./result_alpha.txt', 'r') as file:
#     # Read all lines from the file and store them in a list
#     lines = file.readlines()
# # lines
# alpha_result = []
# for line in lines:
#     alpha_result.append(line.strip())

# with open('./result_alpha_appendix_1_r.txt', 'r') as file:
#     # Read all lines from the file and store them in a list
#     lines = file.readlines()
# # lines
# result_1 = []
# for line in lines:
#     result_1.append(line.strip())
# len(result_1)

# with open('./result_alpha_appendix_2_r.txt', 'r') as file:
#     # Read all lines from the file and store them in a list
#     lines = file.readlines()
# # lines
# result_2 = []
# for line in lines:
#     result_2.append(line.strip())

# with open('./result_alpha_appendix_3_r.txt', 'r') as file:
#     # Read all lines from the file and store them in a list
#     lines = file.readlines()
# # lines
# result_3 = []
# for line in lines:
#     result_3.append(line.strip())

# for app in result_1:
#     index, remain = find_number_and_remainder(app)
#     alpha_result[index] = alpha_result[index] + ' ' + remain
# for app in result_2:
#     index, remain = find_number_and_remainder(app)
#     alpha_result[index] = alpha_result[index] + ' ' + remain
# for app in result_3:
#     index, remain = find_number_and_remainder(app)
#     alpha_result[index] = alpha_result[index] + ' ' + remain

In [9]:
# # Open the text file
# #kllum
# with open('./result_alpha.txt', 'r') as file:
#     # Read all lines from the file and store them in a list
#     lines = file.readlines()
# # lines
# alpha_result = []
# for line in lines:
#     alpha_result.append(line.strip())

# with open('./result_alpha_appendix_1_r.txt', 'r') as file:
#     # Read all lines from the file and store them in a list
#     lines = file.readlines()
# # lines
# result_1 = []
# for line in lines:
#     result_1.append(line.strip())
# len(result_1)

# with open('./result_alpha_appendix_2_r.txt', 'r') as file:
#     # Read all lines from the file and store them in a list
#     lines = file.readlines()
# # lines
# result_2 = []
# for line in lines:
#     result_2.append(line.strip())

# with open('./result_alpha_appendix_3_r.txt', 'r') as file:
#     # Read all lines from the file and store them in a list
#     lines = file.readlines()
# # lines
# result_3 = []
# for line in lines:
#     result_3.append(line.strip())

# for app in result_1:
#     index, remain = find_number_and_remainder(app)
#     alpha_result[index] = alpha_result[index] + ' ' + remain
# for app in result_2:
#     index, remain = find_number_and_remainder(app)
#     alpha_result[index] = alpha_result[index] + ' ' + remain
# for app in result_3:
#     index, remain = find_number_and_remainder(app)
#     alpha_result[index] = alpha_result[index] + ' ' + remain

In [23]:
import llm_blender
blender = llm_blender.Blender()
blender.loadranker("llm-blender/PairRM") # load ranker checkpoint


/usr/local/lib/python3.8/dist-packages/dataclasses_json/core.py:188: RuntimeWarning: 'NoneType' object value of non-optional type load_checkpoint detected when decoding RankerConfig.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/dataclasses_json/core.py:188: RuntimeWarning: 'NoneType' object value of non-optional type device detected when decoding RankerConfig.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:515: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Successfully loaded ranker from  /root/.cache/huggingface/hub/llm-blender/PairRM


In [24]:
# inputs = inputs[:2]
len(inputs)

130

In [26]:
candidates_texts = []
for idx, skt in enumerate(skt_result):
    candidates_texts.append([skt,alpha_result[idx]])
                            


# candidates_texts = [[skt_result[0],alpha_result[0]],[skt_result[1],alpha_result[1]]]
len(candidates_texts)

130

In [13]:
# alpha_result[0]
alpha_result[50]

'답변 네, 고습도 환경에서 몰딩 소재가 팽창하거나 수축하여 변형될 수 있습니다. 따라서, 습도가 높은 공간에 몰딩을 설치할 경우 몰딩 소재의 품질과 내구성을 고려하여 적합한 소재를 선택하는 것이 중요합니다. 또한, 몰딩 설치 전에는 습도 조절에 신경을 쓰고, 몰딩 소재의 건조시간을 충분히 고려해야 합니다. 내외부 온도의 큰 차이는 도배지에 결로를 발생시키고, 이로 인해 곰팡이가 생길 가능성이 있습니다. 따라서 온도 관리가 중요하며, 온도차를 줄이기 위해 냉열기를 가동하는 것이 도움이 될 수 있습니다. 또한, 제습기를 가동하여 실내 습도를 관리하는 것도 중요합니다. 만약 이러한 조치에도 불구하고 곰팡이가 발생한다면, 전문가의 도움을 받아 곰팡이 제거와 취급하는 것이 좋습니다.'

In [12]:
# skt_result[0]
skt_result[50]

'고습도 환경에서 몰딩 소재가 팽창하거나 수축하여 변형될 수 있습니다. 따라서, 습도가 높은 공간에서 몰딩을 사용할 때는 소재의 특성을 고려하여 적합한 소재를 선택하는 것이 중요합니다. 또한, 몰딩을 시공한 후에도 공간의 습도를 적절히 관리하여 변형을 최소화할 필요가 있습니다. 내부와 외부 온도의 큰 차이로 인해 도배지에 결로가 발생하여 곰팡이가 발생할 수 있습니다. 온도 변화로 인해 도배지가 팽창하고 수축할 때 결로가 생기므로, 실내와 외부의 온도차를 줄이는 것이 중요합니다. 추가로, 제습기를 사용하여 실내의 적정 습도를 유지하고 공간을 환기시키는 것이 도움이 될 수 있습니다. 만약 곰팡이 발생이 지속된다면, 전문가의 도움을 받아야 합니다.'

In [32]:
ranks = blender.rank(inputs, candidates_texts, return_scores=False, batch_size=1)
ranks

Ranking candidates: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 130/130 [00:11<00:00, 11.64it/s]


array([[2, 1],
       [1, 2],
       [1, 2],
       [2, 1],
       [2, 1],
       [1, 2],
       [1, 2],
       [2, 1],
       [1, 2],
       [2, 1],
       [2, 1],
       [1, 2],
       [1, 2],
       [1, 2],
       [2, 1],
       [1, 2],
       [2, 1],
       [2, 1],
       [2, 1],
       [1, 2],
       [1, 2],
       [1, 1],
       [2, 1],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [2, 1],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [2, 1],
       [2, 1],
       [2, 1],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [1, 2],
       [2, 1],
       [2, 1],
       [2, 1],
       [2, 1],
       [1, 2],
       [1, 2],
       [2, 1],
       [1, 2],
       [1, 2],
       [2, 1],
       [2, 1],
       [1, 2],
       [2, 1],
       [1, 2],
       [1, 1],
       [2, 1],
       [1, 2],
       [1, 2],
       [1, 2],
       [2, 1],
       [2, 1],
       [2, 1],
       [1, 2],
       [2,

In [50]:
print(len(ranks))
print(len(skt_result))

130
130


In [55]:
ensemble_results = []
for total_idx, rank in enumerate(ranks):
    print(total_idx)
    for idx, r in enumerate(rank):
        if r==1:
            if idx == 0:
                ensemble_results.append(skt_result[total_idx])
            elif idx == 1:
                ensemble_results.append(alpha_result[total_idx])
            else:
                print("error")
            break


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129


In [57]:
print(len(ensemble_results))
ensemble_results
# answer_only = answer_only.replace('\n', ' ')

130


['방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 주로 가격대가 높고 관리가 어렵다는 점입니다. 또한 습기에 약해 뒤틀림, 부서짐, 수축 및 팽창이 발생할 수 있다는 점도 단점으로 꼽힙니다.',
 '도배지에 녹이 발생하는 원인은 공간 내 습도가 높거나 누수가 발생하여 발생할 수 있습니다. 이로 인해 도배지 안쪽의 금속이 녹아서 녹이 도배지에 베어나오게 됩니다. 습기가 높은 상태가 지속될 경우 녹오염이 발생할 수 있으며, 또한 누수에 의해 도배지가 젖은 상태가 유지될 경우 곰팡이가 발생할 수 있습니다.  이러한 경우에는 먼저, 제습기를 가동하거나 환기를 통해 공간 내 적정한 습도를 유지하는 것이 중요합니다. 또한, 누수를 제거하고 곰팡이가 발생한 도배지의 부분 및 전체를 교체하는 것이 필요할 수 있습니다. 이러한 보수작업은 전문가의 도움을 받아 진행하는 것이 좋습니다.',
 '큐블럭은 일반 벽돌에 비해 가격이 비싸고 균열이 생기기 쉬우며 습기로 인해 하자가 발생할 수 있다는 점이 단점으로 꼽힙니다. 이러한 단점으로 인해 섬유 시멘트 블록이나 기타 대안적 소재를 고려하는 경우가 많습니다. 압출법단열판은 습기에 강하고 곰팡이 및 세균 증식을 막을 수 있으며 단열재 중 열전도율이 가장 낮은 편이고 시공이 간편하다는 장점이 있습니다.',
 '철골구조는 화재 발생 시 내구성이 약해지며, 이는 건물의 붕괴로 이어질 수 있습니다. 이러한 문제를 해결하기 위한 방법 중 하나는 내화성 구조물을 사용하는 것입니다. 예를 들어, 벽돌, 콘크리트 또는 석고보드와 같은 내화성 자재로 철골을 둘러싸 화재에 의한 파손을 최소화할 수 있습니다. 또한 화재감지 및 화재진압 장치를 설치하여 화재가 발생했을 때의 초기 대응에 대비하는 것도 중요합니다. 더 나아가 철골재의 피해를 최소화하기 위해 화재 대피 및 구조에 대한 교육, 규제 및 점검 프로토콜을 강화하는 것이 필요합니다.',
 '

In [60]:
 # ensemble_results[0]

'방청페인트의 종류는 광명단페인트, 방청산화철페인트, 알미늄페인트, 역청질페인트, 워시프라이머, 크롬산아연페인트, 규산염페인트가 있습니다. 원목사이딩의 단점은 주로 가격대가 높고 관리가 어렵다는 점입니다. 또한 습기에 약해 뒤틀림, 부서짐, 수축 및 팽창이 발생할 수 있다는 점도 단점으로 꼽힙니다.'

In [62]:
# ensemble_results[0].replace('\n',' ')
preds = []

for ensemble_result in ensemble_results:
    preds.append(ensemble_result.replace('\n',' '))

In [18]:
preds = []

for ensemble_result in alpha_result:
    preds.append(ensemble_result.replace('\n',' '))
len(alpha_result)

130

In [59]:
# !pip install sentence_transformers

In [19]:
from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

# 생성한 모든 응답(답변)으로부터 Embedding Vector 추출
pred_embeddings = model.encode(preds)
pred_embeddings.shape
submit = pd.read_csv('./data/sample_submission.csv')
# 제출 양식 파일(sample_submission.csv)을 활용하여 Embedding Vector로 변환한 결과를 삽입
submit.iloc[:,1:] = pred_embeddings
submit.head()
# 리더보드 제출을 위한 csv파일 생성
submit.to_csv('./alpha.csv', index=False)


In [36]:
# blender.loadfuser("llm-blender/gen_fuser_3b") # load fuser checkpoint if you want to use pre-trained fuser; or you can use ranker only
# from llm_blender.blender.blender_utils import get_topk_candidates_from_ranks
# topk_candidates = get_topk_candidates_from_ranks(ranks, candidates_texts, top_k=3)
# fuse_generations = blender.fuse(inputs, topk_candidates, batch_size=2)
# fuse_generations are the fused generations from our fine-tuned checkpoint

# You can also do the rank and fusion with a single function

# fuse_generations, ranks = blender.rank_and_fuse(inputs, candidates_texts, return_scores=False, batch_size=2, top_k=3)


In [ ]:
blender.loadfuser("llm-blender/gen_fuser_3b") # load fuser checkpoint if you want to use pre-trained fuser; or you can use ranker only
from llm_blender.blender.blender_utils import get_topk_candidates_from_ranks
topk_candidates = get_topk_candidates_from_ranks(ranks, candidates_texts, top_k=3)
fuse_generations = blender.fuse(inputs, topk_candidates, batch_size=2)
# fuse_generations are the fused generations from our fine-tuned checkpoint

# You can also do the rank and fusion with a single function

fuse_generations, ranks = blender.rank_and_fuse(inputs, candidates_texts, return_scores=False, batch_size=2, top_k=3)


In [22]:
inputs = ["hello, how are you!", "I love you!"]
candidates_texts = [["get out!", "hi! I am fine, thanks!", "bye!"], 
                    ["I love you too!", "I hate you!", "Thanks! You're a good guy!"]]
print(len(inputs))
print(len(candidates_texts))

2
2


In [21]:
inputs = ["hello, how are you!", "I love you!"]
candidates_texts = [["get out!", "hi! I am fine, thanks!", "bye!"], 
                    ["I love you too!", "I hate you!", "Thanks! You're a good guy!"]]
ranks = blender.rank(inputs, candidates_texts, return_scores=False, batch_size=1)
# ranks is a list of ranks where ranks[i][j] represents the ranks of candidate-j for input-i
"""
ranks -->
array([[3, 1, 2], # it means "hi! I am fine, thanks!" ranks the 1st, "bye" ranks the 2nd, and "get out!" ranks the 3rd. 
       [1, 3, 2]], # it means "I love you too"! ranks the the 1st, and "I hate you!" ranks the 3rd.
       dtype=int32) 

"""

Ranking candidates: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.70s/it]


'\nranks -->\narray([[3, 1, 2], # it means "hi! I am fine, thanks!" ranks the 1st, "bye" ranks the 2nd, and "get out!" ranks the 3rd. \n       [1, 3, 2]], # it means "I love you too"! ranks the the 1st, and "I hate you!" ranks the 3rd.\n       dtype=int32) \n\n'

In [60]:
# from llm_blender.pair_ranker.pairrm import DebertaV2PairRM # or copy the DebertaV2PairRM definition here, https://github.com/yuchenlin/LLM-Blender/blob/main/llm_blender/pair_ranker/pairrm.py
# from transformers import AutoTokenizer
# from typing import List

# pairrm = DebertaV2PairRM.from_pretrained("llm-blender/PairRM-hf").eval()
# tokenizer = AutoTokenizer.from_pretrained('llm-blender/PairRM-hf')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [61]:
len(inputs)

130

In [62]:
len(skt_result)

130

In [63]:
len(alpha_result)

130

In [9]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from llm_blender.pair_ranker.pairrm import DebertaV2PairRM # or copy the DebertaV2PairRM definition here, https://github.com/yuchenlin/LLM-Blender/blob/main/llm_blender/pair_ranker/pairrm.py
from transformers import AutoTokenizer
from typing import List
pairrm = DebertaV2PairRM.from_pretrained("llm-blender/PairRM-hf")
pairrm.to(device)
pairrm.eval()
tokenizer = AutoTokenizer.from_pretrained('llm-blender/PairRM-hf')
source_prefix = "<|source|>"
cand1_prefix = "<|candidate1|>"
cand2_prefix = "<|candidate2|>"
# inputs = inputs
# candidates_A = skt_result
# candidates_B = alpha_result


def tokenize_pair(sources:List[str], candidate1s:List[str], candidate2s:List[str], source_max_length=1224, candidate_max_length=412):
    ids = []
    assert len(sources) == len(candidate1s) == len(candidate2s)
    max_length = source_max_length + 2 * candidate_max_length
    for i in range(len(sources)):
        source_ids = tokenizer.encode(source_prefix + sources[i], max_length=source_max_length, truncation=True)
        candidate_max_length = (max_length - len(source_ids)) // 2
        candidate1_ids = tokenizer.encode(cand1_prefix + candidate1s[i], max_length=candidate_max_length, truncation=True)
        candidate2_ids = tokenizer.encode(cand2_prefix + candidate2s[i], max_length=candidate_max_length, truncation=True)
        ids.append(source_ids + candidate1_ids + candidate2_ids)
    encodings = tokenizer.pad({"input_ids": ids}, return_tensors="pt", padding="max_length", max_length=max_length)
    return encodings

# encodings = tokenize_pair(inputs, candidates_A, candidates_B)
# encodings = {k:v.to(pairrm.device) for k,v in encodings.items()}
# outputs = pairrm(**encodings)
# logits = outputs.logits.tolist()
# comparison_results = outputs.logits > 0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
inputs_s = inputs[:10]
candidates_A_s = skt_result[:10]
candidates_B_s = alpha_result[:10]

In [11]:
encodings = tokenize_pair(inputs_s, candidates_A_s, candidates_B_s)
encodings = {k:v.to(pairrm.device) for k,v in encodings.items()}
outputs = pairrm(**encodings)
logits = outputs.logits.tolist()
comparison_results = outputs.logits > 0

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [15]:
outputs


SequenceClassifierOutput(loss=None, logits=tensor([  0.6082,   2.0109,   5.3158,  -9.1768,  -7.7675,   8.0936,   0.4748,
        -10.6426,   8.1103,  -9.0774], device='cuda:0',
       grad_fn=<MeanBackward1>), hidden_states=None, attentions=None)

In [12]:
# outputs
comparison_results

tensor([ True,  True,  True, False, False,  True,  True, False,  True, False],
       device='cuda:0')

In [ ]:
print(logits)
# [1.9003021717071533, -1.2547134160995483]
print(comparison_results)
# tensor([ True, False], device='cuda:0'), which means whether candidate A is better than candidate B for each input